In [1]:
import optuna
import numpy as np
# Turn off optuna logging
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Self-defined modules
from config import *

# Loan

In [2]:
def objective(trial):
    residual = loan
    x = trial.suggest_float('x', 0, 1e6)
    for d in range(num_month):    
        residual = residual - x + residual * rate / 12
    return abs(residual)

In [6]:
# Run
study = optuna.create_study()
study.optimize(objective, n_trials=1000)
print(study.best_params)
print(study.best_value)

{'x': 54327.479921546714}
7900.974746351711


# Maintanance

In [4]:
# 1. We need to pay 3300 for maintenance per month
# 2. We need th pay tax for the house by 0.6% of the house price per year
def getHouseBill(maintenance, num_month, house_price=house_price, house_rent=house_rent, cpi_inflation=cpi_inflation):
    ans = 0
    rollout_maintenance = maintenance
    tax = 0.006 * house_price / 12
    rollout_tax = tax
    rollout_house_rent = house_rent
    bill_history = np.zeros((num_month, 4))
    for i in range(num_month):
        ans += rollout_maintenance + rollout_tax
        rollout_maintenance *= (1 + cpi_inflation/12)
        rollout_tax *= (1 + cpi_inflation/12)
        rollout_house_rent = rollout_house_rent * (1 + cpi_inflation/12)
        #print(f"The bill this month is: {ans:.0f} The maintenance is: {rollout_maintenance:.0f} The tax is: {rollout_tax:.0f}")
        bill_history[i, 0] = rollout_maintenance
        bill_history[i, 1] = rollout_tax
        bill_history[i, 2] = rollout_house_rent
    # Save the bills as a table
    np.savetxt('./bills.txt', bill_history, fmt='%.0f', delimiter='\t', header='Maintenance, Tax, HouseRent')
    return ans
main_tax_30y = getHouseBill(management_cost, num_month)
print(main_tax_30y)


6060522.266766507


In [14]:
print("House price: ", house_price)
print("Cash: ", cash)
bill_history = np.loadtxt('./bills.txt', delimiter='\t', skiprows=1)
house_rent_history = bill_history[:, 2]
house_loan_monthly = 54166
total_cost =  np.sum(house_rent_history) + house_loan_monthly * num_month + cash
print("Total cost: ", total_cost)
total_loan = house_loan_monthly*num_month
print("Total loan: ", total_loan)
net_worth = house_price*((1+house_infaltion-decay_rate)**30)
print("Net Worth with decay: ", net_worth)
net_worth = house_price*((1+house_infaltion)**30)
print("Net Worth without decay: ", net_worth)


House price:  18000000.0
Cash:  3600000.0
Total cost:  37906159.0
Total loan:  19499760
Net Worth with decay:  28979023.265623987
Net Worth without decay:  43690724.48141392
